# Metaclasses and Attributes

Metaclasses are often mentioned in lists of Python’s features, but few understand what they accomplish in practice. The name `metaclass` vaguely implies **a concept above and beyond a class**. Simply put, **metaclasses let you intercept Python’s class statement and provide special behavior each time a class is defined**.

Similarly mysterious and powerful are Python’s built-in features for **dynamically customizing attribute accesses**. Along with Python’s object-oriented constructs, these facilities provide wonderful tools to ease the transition from simple classes to complex ones.

However, **with these powers come many pitfalls**. Dynamic attributes enable you to override objects and cause unexpected side effects. Metaclasses can create extremely bizarre behaviors that are unapproachable to newcomers. It’s important that you **follow the rule of least surprise** and **only use these mechanisms to implement well-understood idioms**.

## Item 29: Use Plain Attributes Instead of Get and Set Methods

Programmers coming to Python from other languages may naturally try to implement explicit getter and setter methods.

```python
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms

    def get_ohms(self):
        return self._ohms

    def set_ohms(self, ohms):
        self._ohms = ohms

# OK, but not Pythonic
r0 = OldResistor(50e3)
r0.set_ohms(10e3)

# clumsy
r0.set_ohms(r0.get_ohms() + 5e3)
```

These utility methods do **help define the interface for your class**, making it **easier to encapsulate functionality**, **validate usage**, and **define boundaries**. Those are important goals when designing a class to ensure you don’t break callers as your class evolves over time.

**In Python, however, you almost never need to implement explicit setter or getter methods. Instead, you should always start your implementations with simple public attributes.**

```python
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e3
r1.ohms += 5e3
```

Specifying a `setter` on a property also lets you perform type checking and validation on values passed to ur class.

```python
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._ohms = ohms

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms

r3 = BoundedResistance(1e3)
r3.ohms = 0

>>>
ValueError: 0.000000 ohms must be > 0
```

**The biggest shortcoming of `@property` is that the methods for an attribute can only be shared by subclasses.** Unrelated classes can’t share the same implementation. However, Python also supports **descriptors** that enable reusable property logic and many other use cases.

Finally, when you use `@property` methods to implement setters and getters, **be sure that the behavior you implement is not surprising**. For example, don’t set other attributes in getter property methods.

Things to Remember
* Define new class interfaces using simple public attributes, and avoid set and get methods.
* Use @property to define special behavior when attributes are accessed on your objects, if necessary.
* Follow the rule of least surprise and avoid weird side effects in your @property methods.
* Ensure that @property methods are fast; do slow or complex work using normal methods.

## Item 30: Consider @property Instead of Refactoring Attributes

One advanced but common use of @property is **transitioning what was once a simple numerical attribute into an on-the-fly calculation.** This is extremely helpful because it lets you migrate all existing usage of a class to have new behaviors without rewriting any of the call sites. It also provides an important stopgap for improving your interfaces over time.

`@property` is a tool to help you address problems you’ll come across in real-world code. Don’t overuse it. When you find yourself repeatedly extending `@property` methods, it’s probably time to refactor your class instead of further paving over your code’s poor design.

Things to Remember
* Use @property to give existing instance attributes new functionality.
* Make incremental progress toward better data models by using `@property`
* Consider refactoring a class and all call sites when you find yourself using `@property` too heavily

## Item 31: Use Descriptors for Reusable `@property` Methods



## Item 32: Use `__getattr__`, `__getattribute__`, and `__setattr__` for Lazy Attributes

Python’s language hooks make it easy to write generic code for gluing systems together. For example, say you want to represent the rows of your database as Python objects. Your database has its schema set. Your code that uses objects corresponding to those rows must also know what your database looks like. However, in Python, the code that connects your Python objects to the database doesn’t need to know the schema of your rows; it can be generic.

Python makes this dynamic behavior possible with the `__getattr__` special method, if ur class defines `__getattr__`, **that method is called every time and attribute can't be found in an object's instance dictionary**.

```python
class LazyDB(object):
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value

data = LazyDB()
print('Before: ', data.__dict__)
print('foo', data.foo)
print('After: ', data.__dict__)

>>>
Before:  {'exists': 5}
foo Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}
```

This behavior is especially helpful for use cases like lazily accessing schemaless data. `__getattr__` runs once to do the hard work of loading a property; all subsequent accesses retrieve the existing result.

Python has another language hook called `__getattribute__`. **This special method is called every time an attribute is accessed on an object, even in cases where it does exist** in the attribute dictionary. This enables you to do things like check global transaction state on every property access.

```python
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            print('Added (%s) attr' % name)
            return value

data = ValidatingDB()
print('exists: ', data.exists)
print('foo: ', data.foo)
print('foo: ', data.foo)

>>>
exists:  5
Called __getattribute__(foo)
Added (foo) attr
foo:  Value for foo
Called __getattribute__(foo)
foo:  Value for foo
```

The `__setattr__` method is always called every time an attribute is assigned on an instance.

Things to Remember
* Use `__getattr__` and `__setattr__` to lazily load and save attributes for an object.
* Understand that `__getattr__` only gets called once when accessing a missing attribute, whereas `__getattribute__` gets called every time an attribute is accessed. 
* Avoid infinite recursion in `__getattribute__` and `__setattr__` by using methods from super() (i.e., the object class) to access instance attributes directly.


## Item 33: Validate Subclasses with Metaclasses

One of the simplest applications of metaclasses is **verifying that a class was defined correctly**. When you’re building a complex class hierarchy, you may want to enforce style, require overriding methods, or have strict relationships between class attributes. Metaclasses enable these use cases by **providing a reliable way to run your validation code each time a new subclass is defined**.

Often a class’s validation code runs in the `__init__` method, when an object of the class’s type is constructed. Using metaclasses for validation can raise errors much earlier.

Before I get into how to define a metaclass for validating subclasses, it’s important to understand the metaclass action for standard objects. **A metaclass is defined by inheriting from `type`**. In the default case, **a metaclass receives the contents of associated class statements in its `__new__` method**.

```python
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print((meta, name, bases, class_dict))
        return type.__new__(meta, name, bases, class_dict)


class MyClass(object, metaclass=Meta):
    stuff = 123

    def foo(self):
        pass
```

When you `%run` this file, you will find the `__new__` is executed, so we can add functionality to it in order to validate all of the params of a class before it's defined.

```
(<class '__main__.Meta'>, 'MyClass', (<class 'object'>,), {'__qualname__': 'MyClass', '__module__': '__main__', 'foo': <function MyClass.foo at 0x103c24bf8>, 'stuff': 123})
```

Things to Remember
* Use metaclasses to ensure that subclasses are well formed at the time they are defined, before objects of their type are constructed.
* Metaclasses have slightly different syntax in Python 2 vs. Python 3.
* The `__new__` method of metaclass is run after the `class` statement's entire body has been processed.

## Item 34: Register Class Existence with Metaclasses

Another common use of metaclasses is to automatically register types in your program. Registration is useful for doing reverse lookups, where you need to map a simple identifier back to a corresponding class.

For example, say you want to implement your own serialized representation of a Python object using JSON. You need a way to take an object and turn it into a JSON string. Here, I do this generically by defining a base class that records the constructor parameters and turns them into a JSON dictionary:

```python
import json


class Serializable(object):
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({'args': self.args})


class Deserializable(Serializable):
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params['args'])


class BetterPoint2D(Deserializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    def __repr__(self):
        return 'BetterPoint2D(%d, %d)' % (self.x, self.y)


point = BetterPoint2D(5, 3)
print('Before: ', point)
data = point.serialize()
print('Serialized: ', data)
after = BetterPoint2D.deserialize(data)
print('After: ', after)

>>>
Before:  BetterPoint2D(5, 3)
Serialized:  {"args": [5, 3]}
After:  BetterPoint2D(5, 3)
```

The problem with this approach is that it only works if you know the intended type of the serialized data ahead of time(e.g. Point2D, BetterPoint2D). **Ideally, you'd have a large number of classes serializing to JSON and one common function that could deserialize any of them back to a corresponding Py obj.**

```python
import json


class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args': self.args,
        })


registry = {}


def register_class(target_class):
    registry[target_class.__name__] = target_class


def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])


class EvenBetterPoint2D(BetterSerializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    def __repr__(self):
        return 'EvenBetterPoint2D(%d, %d)' % (self.x, self.y)

register_class(EvenBetterPoint2D)


point = EvenBetterPoint2D(5, 3)
print('Before: ', point)
data = point.serialize()
print('Serialized: ', data)
after = deserialize(data)
print('After: ', after)
```

The second version is better, but you still need to register every class you need to serialize and deserialize. **Metaclasses** enable this by intercepting the `class` statement when subclasses are defined. **This lets you register the new type immediately after the class's body**.

```python
import json


class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args': self.args,
        })


registry = {}


def register_class(target_class):
    registry[target_class.__name__] = target_class


def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])


class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls


class RegisteredSerializable(BetterSerializable, metaclass=Meta):
    pass


class Vector3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x = x
        self.y = y
        self.z = z

    def __repr__(self):
        return 'Vector3D(%d, %d, %d)' % (self.x, self.y, self.z)


point = Vector3D(10, -7, 3)
print('Before: ', point)
data = point.serialize()
print('Serialized: ', data)
after = deserialize(data)
print('After: ', after)
```

Using metaclasses for class registration ensures that you’ll never miss a class as long as the inheritance tree is right. This works well for serialization, as I’ve shown, and also applies to database object-relationship mappings (ORMs), plug-in systems, and system hooks.

Things to Remember
* Class registration is a helpful pattern for building modular Python programs.
* Metaclasses let you run registration code automatically each time your base class is subclassed in a program.
* Using metaclasses for class registration avoids errors by ensuring that you never miss a registration call.

## Item 35: Annotate Class Attributes with Metaclasses

